# **Data Preprocessing of Cell2Cell Dataset**

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

data = [pd.read_csv("dataset_cell2cell_train.csv"), pd.read_csv("dataset_cell2cell_holdout.csv")]

## **Preprocess Train Set**

### **Checking Dataset Properties**

In [2]:
data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51047 entries, 0 to 51046
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CustomerID                 51047 non-null  int64  
 1   Churn                      51047 non-null  object 
 2   MonthlyRevenue             50891 non-null  float64
 3   MonthlyMinutes             50891 non-null  float64
 4   TotalRecurringCharge       50891 non-null  float64
 5   DirectorAssistedCalls      50891 non-null  float64
 6   OverageMinutes             50891 non-null  float64
 7   RoamingCalls               50891 non-null  float64
 8   PercChangeMinutes          50680 non-null  float64
 9   PercChangeRevenues         50680 non-null  float64
 10  DroppedCalls               51047 non-null  float64
 11  BlockedCalls               51047 non-null  float64
 12  UnansweredCalls            51047 non-null  float64
 13  CustomerCareCalls          51047 non-null  flo

In [3]:
print(data[0].head(25))

    CustomerID Churn  MonthlyRevenue  MonthlyMinutes  TotalRecurringCharge  \
0      3000002   Yes           24.00           219.0                  22.0   
1      3000010   Yes           16.99            10.0                  17.0   
2      3000014    No           38.00             8.0                  38.0   
3      3000022    No           82.28          1312.0                  75.0   
4      3000026   Yes           17.14             0.0                  17.0   
5      3000030    No           38.05           682.0                  52.0   
6      3000038    No           31.66            26.0                  30.0   
7      3000042    No           62.13            98.0                  66.0   
8      3000046    No           35.30            24.0                  35.0   
9      3000050    No           81.00          1056.0                  75.0   
10     3000054    No           25.23             2.0                  25.0   
11     3000058    No          212.51          1972.0            

In [4]:
data[0].isnull().sum()

CustomerID                     0
Churn                          0
MonthlyRevenue               156
MonthlyMinutes               156
TotalRecurringCharge         156
DirectorAssistedCalls        156
OverageMinutes               156
RoamingCalls                 156
PercChangeMinutes            367
PercChangeRevenues           367
DroppedCalls                   0
BlockedCalls                   0
UnansweredCalls                0
CustomerCareCalls              0
ThreewayCalls                  0
ReceivedCalls                  0
OutboundCalls                  0
InboundCalls                   0
PeakCallsInOut                 0
OffPeakCallsInOut              0
DroppedBlockedCalls            0
CallForwardingCalls            0
CallWaitingCalls               0
MonthsInService                0
UniqueSubs                     0
ActiveSubs                     0
ServiceArea                   24
Handsets                       1
HandsetModels                  1
CurrentEquipmentDays           1
AgeHH1    

### **Identifying Type of Features**

In [5]:
numericalData = [data[0].select_dtypes(include = ['float64']).columns,
                 data[0].select_dtypes(include = ['int64']).columns]
print(f"\nNumerical Features (float):\n{numericalData[0]}")
print(f"\nNumerical Features (int):\n{numericalData[1]}")

categoricalData = data[0].select_dtypes(include = ['object']).columns
print(f"\nCategorical Features:\n{categoricalData}")


Numerical Features (float):
Index(['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
       'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls',
       'ReceivedCalls', 'OutboundCalls', 'InboundCalls', 'PeakCallsInOut',
       'OffPeakCallsInOut', 'DroppedBlockedCalls', 'CallForwardingCalls',
       'CallWaitingCalls', 'Handsets', 'HandsetModels', 'CurrentEquipmentDays',
       'AgeHH1', 'AgeHH2'],
      dtype='object')

Numerical Features (int):
Index(['CustomerID', 'MonthsInService', 'UniqueSubs', 'ActiveSubs',
       'RetentionCalls', 'RetentionOffersAccepted',
       'ReferralsMadeBySubscriber', 'IncomeGroup',
       'AdjustmentsToCreditRating'],
      dtype='object')

Categorical Features:
Index(['Churn', 'ServiceArea', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeowners

### **Handling Missing Data**

In [6]:
# Drop any entries with missing value for certain columns

columns = ['Handsets', 'HandsetModels', 'CurrentEquipmentDays'] 
data[0] = data[0].dropna(subset=columns)

In [7]:
# Fill missing values with the mean of the column

columns = ['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge', 'DirectorAssistedCalls',
           'OverageMinutes', 'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues'] 

for i in range(len(columns)):
    data[0][columns[i]].fillna(data[0][columns[i]].mean(), inplace=True)

In [8]:
# Fill missing values with string 'UNKNOWN'

data[0]['ServiceArea'] = data[0]['ServiceArea'].fillna("UNKNOWN")

In [9]:
# Fill missing values with 0

data[0]['AgeHH1'] = data[0]['AgeHH1'].fillna(0)
data[0]['AgeHH2'] = data[0]['AgeHH2'].fillna(0)

### **Modifying the Datatypes**

In [10]:
# Changing dtype of Customer ID from int to object 

data[0]['CustomerID'] = data[0]['CustomerID'].astype('object')

In [11]:
# Manually do the One Hot Encoding (idk)

mapping = {'Yes' : 1, 'No' : 0}

columns = ['Churn', 'ChildrenInHH', 'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner',
           'RVOwner', 'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
           'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
           'NotNewCellphoneUser', 'OwnsMotorcycle', 'MadeCallToRetentionTeam'] 

for i in range(len(columns)):
    data[0][columns[i]] = data[0][columns[i]].map(mapping)

In [12]:
# Changing credit rating dtype from object to int, scale 1 to 5

credRate = {'1-Highest' : 1, '2-High' : 2, '3-Good' : 3, '4-Medium' : 4,
            '5-Low' : 5, '6-VeryLow' : 6, '7-Lowest' : 7}

data[0]['CreditRating'] = data[0]['CreditRating'].map(credRate)

### **Dataset After Preprocessed**

In [13]:
data[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51046 entries, 0 to 51046
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CustomerID                 51046 non-null  object 
 1   Churn                      51046 non-null  int64  
 2   MonthlyRevenue             51046 non-null  float64
 3   MonthlyMinutes             51046 non-null  float64
 4   TotalRecurringCharge       51046 non-null  float64
 5   DirectorAssistedCalls      51046 non-null  float64
 6   OverageMinutes             51046 non-null  float64
 7   RoamingCalls               51046 non-null  float64
 8   PercChangeMinutes          51046 non-null  float64
 9   PercChangeRevenues         51046 non-null  float64
 10  DroppedCalls               51046 non-null  float64
 11  BlockedCalls               51046 non-null  float64
 12  UnansweredCalls            51046 non-null  float64
 13  CustomerCareCalls          51046 non-null  flo

In [14]:
print(data[0].head(25))

   CustomerID  Churn  MonthlyRevenue  MonthlyMinutes  TotalRecurringCharge  \
0     3000002      1           24.00           219.0                  22.0   
1     3000010      1           16.99            10.0                  17.0   
2     3000014      0           38.00             8.0                  38.0   
3     3000022      0           82.28          1312.0                  75.0   
4     3000026      1           17.14             0.0                  17.0   
5     3000030      0           38.05           682.0                  52.0   
6     3000038      0           31.66            26.0                  30.0   
7     3000042      0           62.13            98.0                  66.0   
8     3000046      0           35.30            24.0                  35.0   
9     3000050      0           81.00          1056.0                  75.0   
10    3000054      0           25.23             2.0                  25.0   
11    3000058      0          212.51          1972.0            

In [15]:
data[0].isnull().sum()

CustomerID                   0
Churn                        0
MonthlyRevenue               0
MonthlyMinutes               0
TotalRecurringCharge         0
DirectorAssistedCalls        0
OverageMinutes               0
RoamingCalls                 0
PercChangeMinutes            0
PercChangeRevenues           0
DroppedCalls                 0
BlockedCalls                 0
UnansweredCalls              0
CustomerCareCalls            0
ThreewayCalls                0
ReceivedCalls                0
OutboundCalls                0
InboundCalls                 0
PeakCallsInOut               0
OffPeakCallsInOut            0
DroppedBlockedCalls          0
CallForwardingCalls          0
CallWaitingCalls             0
MonthsInService              0
UniqueSubs                   0
ActiveSubs                   0
ServiceArea                  0
Handsets                     0
HandsetModels                0
CurrentEquipmentDays         0
AgeHH1                       0
AgeHH2                       0
Children

## **Preprocess Holdout Set**

### **Checking Dataset Properties**

In [16]:
data[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CustomerID                 20000 non-null  int64  
 1   Churn                      0 non-null      float64
 2   MonthlyRevenue             19940 non-null  float64
 3   MonthlyMinutes             19940 non-null  float64
 4   TotalRecurringCharge       19940 non-null  float64
 5   DirectorAssistedCalls      19940 non-null  float64
 6   OverageMinutes             19940 non-null  float64
 7   RoamingCalls               19940 non-null  float64
 8   PercChangeMinutes          19865 non-null  float64
 9   PercChangeRevenues         19865 non-null  float64
 10  DroppedCalls               20000 non-null  float64
 11  BlockedCalls               20000 non-null  float64
 12  UnansweredCalls            20000 non-null  float64
 13  CustomerCareCalls          20000 non-null  flo

In [17]:
print(data[1].head(25))

    CustomerID  Churn  MonthlyRevenue  MonthlyMinutes  TotalRecurringCharge  \
0      3000006    NaN           57.49           483.0                  37.0   
1      3000018    NaN           55.23           570.0                  72.0   
2      3000034    NaN           97.34          1039.0                  50.0   
3      3000070    NaN           35.59           153.0                  30.0   
4      3000074    NaN           55.27          1213.0                  50.0   
5      3000086    NaN           60.19          1424.0                  56.0   
6      3000098    NaN           25.49           300.0                  25.0   
7      3000110    NaN           37.66           972.0                  36.0   
8      3000246    NaN           42.88           406.0                  40.0   
9      3000254    NaN          342.86          2961.0                 140.0   
10     3000258    NaN           54.96           244.0                  52.0   
11     3000274    NaN           50.48           380.

In [18]:
data[1].isnull().sum()

CustomerID                       0
Churn                        20000
MonthlyRevenue                  60
MonthlyMinutes                  60
TotalRecurringCharge            60
DirectorAssistedCalls           60
OverageMinutes                  60
RoamingCalls                    60
PercChangeMinutes              135
PercChangeRevenues             135
DroppedCalls                     0
BlockedCalls                     0
UnansweredCalls                  0
CustomerCareCalls                0
ThreewayCalls                    0
ReceivedCalls                    0
OutboundCalls                    0
InboundCalls                     0
PeakCallsInOut                   0
OffPeakCallsInOut                0
DroppedBlockedCalls              0
CallForwardingCalls              0
CallWaitingCalls                 0
MonthsInService                  0
UniqueSubs                       0
ActiveSubs                       0
ServiceArea                      4
Handsets                         0
HandsetModels       

### **Identifying Type of Features**

In [19]:
numericalData = [data[1].select_dtypes(include = ['float64']).columns,
                 data[1].select_dtypes(include = ['int64']).columns]
print(f"\nNumerical Features (float):\n{numericalData[0]}")
print(f"\nNumerical Features (int):\n{numericalData[1]}")

categoricalData = data[1].select_dtypes(include = ['object']).columns
print(f"\nCategorical Features:\n{categoricalData}")


Numerical Features (float):
Index(['Churn', 'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
       'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls',
       'ReceivedCalls', 'OutboundCalls', 'InboundCalls', 'PeakCallsInOut',
       'OffPeakCallsInOut', 'DroppedBlockedCalls', 'CallForwardingCalls',
       'CallWaitingCalls', 'AgeHH1', 'AgeHH2'],
      dtype='object')

Numerical Features (int):
Index(['CustomerID', 'MonthsInService', 'UniqueSubs', 'ActiveSubs', 'Handsets',
       'HandsetModels', 'CurrentEquipmentDays', 'RetentionCalls',
       'RetentionOffersAccepted', 'ReferralsMadeBySubscriber', 'IncomeGroup',
       'AdjustmentsToCreditRating'],
      dtype='object')

Categorical Features:
Index(['ServiceArea', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership',
 

### **Handling Missing Data**

In [20]:
# Fill missing values with the mean of the column

columns = ['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge', 'DirectorAssistedCalls',
           'OverageMinutes', 'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues'] 

for i in range(len(columns)):
    data[1][columns[i]].fillna(data[1][columns[i]].mean(), inplace=True)

In [21]:
# Fill missing values with string 'UNKNOWN'

data[1]['ServiceArea'] = data[1]['ServiceArea'].fillna("UNKNOWN")

In [22]:
# Fill missing values with 0

data[1]['AgeHH1'] = data[1]['AgeHH1'].fillna(0)
data[1]['AgeHH2'] = data[1]['AgeHH2'].fillna(0)

### **Modifying the Datatypes**

In [23]:
# Changing dtype of Customer ID from int to object 

data[1]['CustomerID'] = data[1]['CustomerID'].astype('object')

In [24]:
# Manually do the One Hot Encoding (idk)

mapping = {'Yes' : 1, 'No' : 0}

columns = ['ChildrenInHH', 'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner',
           'RVOwner', 'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
           'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
           'NotNewCellphoneUser', 'OwnsMotorcycle', 'MadeCallToRetentionTeam'] 

for i in range(len(columns)):
    data[1][columns[i]] = data[1][columns[i]].map(mapping)

In [25]:
# Changing credit rating dtype from object to int, scale 1 to 5

credRate = {'1-Highest' : 1, '2-High' : 2, '3-Good' : 3, '4-Medium' : 4,
            '5-Low' : 5, '6-VeryLow' : 6, '7-Lowest' : 7}

data[1]['CreditRating'] = data[1]['CreditRating'].map(credRate)

### **Dataset After Preprocessed**

In [26]:
data[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CustomerID                 20000 non-null  object 
 1   Churn                      0 non-null      float64
 2   MonthlyRevenue             20000 non-null  float64
 3   MonthlyMinutes             20000 non-null  float64
 4   TotalRecurringCharge       20000 non-null  float64
 5   DirectorAssistedCalls      20000 non-null  float64
 6   OverageMinutes             20000 non-null  float64
 7   RoamingCalls               20000 non-null  float64
 8   PercChangeMinutes          20000 non-null  float64
 9   PercChangeRevenues         20000 non-null  float64
 10  DroppedCalls               20000 non-null  float64
 11  BlockedCalls               20000 non-null  float64
 12  UnansweredCalls            20000 non-null  float64
 13  CustomerCareCalls          20000 non-null  flo

In [27]:
print(data[1].head(25))

   CustomerID  Churn  MonthlyRevenue  MonthlyMinutes  TotalRecurringCharge  \
0     3000006    NaN           57.49           483.0                  37.0   
1     3000018    NaN           55.23           570.0                  72.0   
2     3000034    NaN           97.34          1039.0                  50.0   
3     3000070    NaN           35.59           153.0                  30.0   
4     3000074    NaN           55.27          1213.0                  50.0   
5     3000086    NaN           60.19          1424.0                  56.0   
6     3000098    NaN           25.49           300.0                  25.0   
7     3000110    NaN           37.66           972.0                  36.0   
8     3000246    NaN           42.88           406.0                  40.0   
9     3000254    NaN          342.86          2961.0                 140.0   
10    3000258    NaN           54.96           244.0                  52.0   
11    3000274    NaN           50.48           380.0            

In [28]:
data[1].isnull().sum()

CustomerID                       0
Churn                        20000
MonthlyRevenue                   0
MonthlyMinutes                   0
TotalRecurringCharge             0
DirectorAssistedCalls            0
OverageMinutes                   0
RoamingCalls                     0
PercChangeMinutes                0
PercChangeRevenues               0
DroppedCalls                     0
BlockedCalls                     0
UnansweredCalls                  0
CustomerCareCalls                0
ThreewayCalls                    0
ReceivedCalls                    0
OutboundCalls                    0
InboundCalls                     0
PeakCallsInOut                   0
OffPeakCallsInOut                0
DroppedBlockedCalls              0
CallForwardingCalls              0
CallWaitingCalls                 0
MonthsInService                  0
UniqueSubs                       0
ActiveSubs                       0
ServiceArea                      0
Handsets                         0
HandsetModels       

## **Exporting Preprocessed Data**

In [29]:
# Export each dataset to a separate CSV file

filename = ['train', 'holdout']

for i, df in enumerate(data):
    df.to_csv(f'preprocessed_data_{filename[i]}.csv', index=False)
    print(f"Data {filename[i]} has been exported to preprocessed_data_{filename[i]}.csv")

Data train has been exported to preprocessed_data_train.csv
Data holdout has been exported to preprocessed_data_holdout.csv
